Framework: 

Comparing Commodities & Synthetic Commodities (Stocks/Indices) During Financial Crises
This framework combines physical commodities (gold, oil, wheat) with synthetic commodities (stocks, ETFs, country indices) to identify the best investment during a crisis.
Time 

In [ ]:
import nasdaq_data_link
import pandas as pd

# Set your API Key
nasdaq_data_link.ApiConfig.api_key = 'NASDAK_DATA_API_KEY'

# Define historical periods
ranges = [("2007-01-01", "2010-12-31"), ("2019-01-01", "2022-12-31")]

def get_quandl_data(code):
    dfs = [nasdaq_data_link.get(code, start_date=start, end_date=end) for start, end in ranges]
    return pd.concat(dfs)

# 1. Spot Prices
gold_spot = get_quandl_data("LBMA/GOLD")
silver_spot = get_quandl_data("LBMA/SILVER")
platinum_spot = get_quandl_data("JOHNMATT/PL")
palladium_spot = get_quandl_data("JOHNMATT/PD")

# 2. Futures Prices (COMEX Continuous Contracts)
gold_futures = get_quandl_data("CHRIS/CME_GC1")
silver_futures = get_quandl_data("CHRIS/CME_SI1")
platinum_futures = get_quandl_data("CHRIS/CME_PL1")
palladium_futures = get_quandl_data("CHRIS/CME_PA1")

# 3. CPI for Inflation Adjustment
cpi = get_quandl_data("RATEINF/CPI_USA")

# 4. Inflation-Adjusted Gold Price
gold_real = gold_spot[['USD (PM)']].dropna().join(cpi, how='inner')
gold_real['Gold Real Price'] = gold_real['USD (PM)'] / (gold_real['Value'] / 100)

# 5. Futures Trading Volume (as liquidity proxy)
futures_volumes = pd.DataFrame({
    'Gold Futures Volume': gold_futures['Volume'],
    'Silver Futures Volume': silver_futures['Volume'],
    'Platinum Futures Volume': platinum_futures['Volume'],
    'Palladium Futures Volume': palladium_futures['Volume']
})

# Save to CSVs
gold_spot.to_csv("gold_spot.csv")
silver_spot.to_csv("silver_spot.csv")
platinum_spot.to_csv("platinum_spot.csv")
palladium_spot.to_csv("palladium_spot.csv")

gold_futures.to_csv("gold_futures.csv")
silver_futures.to_csv("silver_futures.csv")
platinum_futures.to_csv("platinum_futures.csv")
palladium_futures.to_csv("palladium_futures.csv")

gold_real.to_csv("gold_real_adjusted.csv")
futures_volumes.to_csv("futures_volumes.csv")

print("✅ Historical data (2007–2010 & 2019–2022) downloaded from Nasdaq Data Link.")
